In [ ]:
# import the library
%matplotlib inline

import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

# the more advanced python visualization library
import seaborn as sns

# apply style to all the charts
sns.set_style('whitegrid')

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

# convert scientific notation to decimals
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
# https://www.capitalbikeshare.com/system-data
df = pd.read_csv('data/2011-capitalbikeshare-tripdata.csv')
df.head(10)

In [ ]:
# set the datetime column as the index
df['Start date'] = pd.to_datetime(df['Start date'])
df.set_index('Start date', inplace=True)
df.head(10)

# Resampling

In [ ]:
# Resampling daily
df_daily = df.resample('D').count()
df_daily.columns = ['trips']
# TODO: Resampling weekly

# TODO: Resampling monthly


In [ ]:
# plot daily chart
plt.plot(df_daily)
plt.title('Daily')
plt.show()

In [ ]:
# TODO: plot weekly chart


In [ ]:
# TODO: plot monthly chart


# Autocorrelation and Partial Autocorrelation

In [ ]:
# Daily
lags = 52
# acf
plot_acf(df_daily, lags=lags)
plt.show()
# pacf
plot_pacf(df_daily, lags=lags)
plt.show()

In [ ]:
# Weekly
lags = 52
# acf
plot_acf(df_weekly, lags=lags)
plt.show()
# pacf
plot_pacf(df_weekly, lags=lags)
plt.show()

# Differencing 1st order

In [ ]:
df_daily.diff()

In [ ]:
# plot the daily diff
plt.plot(df_daily.diff())
plt.title('Daily Diff')
plt.show()

In [ ]:
# TODO: What is the day with the highest absolute difference?

In [ ]:
# plot the weekly diff
plt.plot(df_weekly.diff())
plt.title('Weekly Diff')
plt.show()

# Time Series Decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from pylab import rcParams
print(rcParams['figure.figsize'])
rcParams['figure.figsize'] = 16, 10
series = df_daily['trips'].dropna().values
result = seasonal_decompose(series, model='additive', freq=7)
result.plot()
plt.show()
rcParams['figure.figsize'] = 6, 4

In [ ]:
df_daily['dow'] = df_daily.index.dayofweek
df_daily['trend'] = result.trend
df_daily['seasonal'] = result.seasonal
df_daily['resid'] = result.resid
df_daily

# Smoothing

In [ ]:
# rolling - moving average
plt.figure(figsize=(10,6))
plt.title('Smoothing Moving Average')
plt.plot(df_daily['trips'], alpha=0.3)
plt.plot(df_daily['trips'].rolling(window=7).mean())
plt.show()

In [ ]:
# exponential weighted functions
plt.figure(figsize=(10,6))
plt.title('Smoothing Exponential Weighted Function')
plt.plot(df_daily['trips'], alpha=0.3)
plt.plot(df_daily['trips'].ewm(alpha=0.3).mean())
plt.show()

# Simple anomaly detection

In [ ]:
resid_mean = df_daily['resid'].dropna().mean()
resid_std = df_daily['resid'].dropna().std()
print(resid_mean)
print(resid_std)

n_std = 3
df_anomaly = df_daily[(df_daily['resid'] < resid_mean - n_std * resid_std)|(df_daily['resid'] > resid_mean + n_std * resid_std)]

plt.figure(figsize=(10,6))
plt.plot(df_daily['trips'])
for k, v in df_anomaly.iterrows():
    plt.plot(k, v['trips'], 'or')
plt.show()

# Time series prediction

## Stats approach

In [ ]:
# Auto Regressive Model
from statsmodels.tsa.ar_model import AR
model = AR(df_daily['trips'])
# fit model
model_fit = model.fit()
print(df_daily['trips'].tail(7))
# calculate prediction
yhat = model_fit.predict(len(df_daily['trips']), len(df_daily['trips'])+14)
print(yhat)
# plot results
plt.figure(figsize=(10,6))
plt.title('Auto Regressive Model')
plt.plot(df_daily['trips'])
plt.plot(yhat)
plt.show()

In [ ]:
# Moving Average
from statsmodels.tsa.arima_model import ARMA
# fit model
model = ARMA(df_daily['trips'], order=(0, 1))
model_fit = model.fit()
# calculate prediction
yhat = model_fit.predict(len(df_daily['trips']), len(df_daily['trips'])+14)
print(yhat)
# plot results
plt.figure(figsize=(10,6))
plt.title('Moving Average Model')
plt.plot(df_daily['trips'])
plt.plot(yhat)
plt.show()

In [ ]:
# Auto Regressive Moving Average
from statsmodels.tsa.arima_model import ARMA
# fit model
model = ARMA(df_daily['trips'], order=(1, 1))
model_fit = model.fit()
# calculate prediction
yhat = model_fit.predict(len(df_daily['trips']), len(df_daily['trips'])+14)
print(yhat)
# plot results
plt.figure(figsize=(10,6))
plt.title('Auto Regressive Moving Average Model')
plt.plot(df_daily['trips'])
plt.plot(yhat)
plt.show()

In [ ]:
# Holt Winters 
from statsmodels.tsa.holtwinters import ExponentialSmoothing
# fit model
model = ExponentialSmoothing(df_daily['trips'], trend='add', seasonal='add', seasonal_periods=7)
model_fit = model.fit(smoothing_level=0.2, smoothing_slope=0.1, smoothing_seasonal=0.5)
# calculate prediction
yhat = model_fit.predict(len(df_daily['trips']), len(df_daily['trips'])+14)
print(yhat)
plt.figure(figsize=(10,6))
plt.title('Holt Winters Model')
plt.plot(df_daily['trips'])
plt.plot(yhat)
plt.show()

## Machine Learning approach

In [ ]:
# Problem definition
# Load the data
# Feature Engineering
# Model Training
# Model Evaluation

## Problem definition
Predict the system usage for tomorrow

## Load the data

In [ ]:
# https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset
df_day = pd.read_csv('data/day.csv')
df_day.head()

In [ ]:
# Convert the date and set as the index
df_day['dteday'] = pd.to_datetime(df_day['dteday'])
df_day.set_index('dteday', inplace=True)
df_day.head(10)

## Feature Engineering

In [ ]:
# Shift example
df_cnt = df_day[['cnt']]
df_cnt['shift_1'] = df_cnt['cnt'].shift(1)
df_cnt.head(10)

In [ ]:
# TODO: apply shift for X days
df_cnt = df_day[['cnt']]


In [ ]:
# Handle Missing Values


In [ ]:
# feature engineering
X_columns = [x for x in df_cnt.columns if x != 'cnt']
y_column = ['cnt']

## Model Training

In [ ]:
# split the data

# do NOT shuffle the data
#df = df.sample(frac=1)

threshold = 0.9
absolute_threshold = int(len(df_cnt)*threshold)
print(len(df_cnt), absolute_threshold)

X_train = df_cnt.iloc[:absolute_threshold][X_columns]
y_train = df_cnt.iloc[:absolute_threshold][y_column]

X_test = df_cnt.iloc[absolute_threshold:][X_columns]
y_test = df_cnt.iloc[absolute_threshold:][y_column]

print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

In [ ]:
# TODO: Train a linear regression or other model


In [ ]:
# TODO: Calculate the MAE


In [ ]:
# plot the results
plt.scatter(y_test, y_pred, alpha=0.3)
plt.show()
plt.plot(y_test.index, y_test)
plt.plot(y_test.index, y_pred)
plt.show()

In [ ]:
# TODO: add the 'atemp' and other features to the list. What is the impact on the MAE?